In [1]:
import pandas as pd
import sqlite3 

# Connexion à la base SQLite
conn = sqlite3.connect("fabric_sim.db")

def load_all(conn):
    """Charge les tables principales avec des jointures logiques"""
    salary = pd.read_sql("SELECT * FROM salary_statement", conn)
    absences = pd.read_sql("SELECT * FROM absence", conn)
    contract = pd.read_sql("SELECT * FROM contract_basis", conn)
    workplan = pd.read_sql("SELECT * FROM work_plan", conn)

    df = salary.merge(absences, on="employee_id", how="left")
    df = df.merge(contract, on="employee_id", how="left")
    df = df.merge(workplan, on="employee_id", how="left")

    # Nettoyage
    df = df.dropna(subset=["total_salary", "planned_days"])  # garde les lignes complètes
    df['absence_rate'] = df['absence_days'] / df['planned_days']

    return df



In [ ]:
# --- Mesures sémantiques ---

def get_total_salary(df):
    return df['total_salary'].sum()

def get_average_absence_rate(df):
    return df['absence_rate'].mean()

def get_salary_by_department(df):
    return df.groupby("department")[["total_salary"]].sum().reset_index()

def get_absence_rate_by_department(df):
    return df.groupby("department")[["absence_rate"]].mean().reset_index()

def filter_by_contract(df, contract_type="CDI"):
    return df[df['contract_type'] == contract_type]
